## Notes 
### Required installing Oracle JAVA 8 to get javabridge installed
### Then, I was able to install py-causal from https://bd2kccd.github.io/docs/py-causal/
### GFCI is slower than RFCI, but more accurate (SPIRTES), GFCI and RFCI account for unobserved variables, FGES assumes no unobserved variables.

Structure Learning Performance Guarantees If the assumptions in the previous section hold, then in the large sample limit, the CBN structure output by GFCId will contain an edge of one of four kinds between Xand Y   if and only if Xand Yare not independent conditional on any subset of the other measured variables of less than or equal to a specified size. In addition, there is (1) an arc X->Y   if and only if Xdirectly or indirectly causes Y, and Y   does not directly or indirectly cause X; (2) an edge X <-->Y   if and only if X   is not a direct or indirect cause of Yand Y   is not a direct or indirect cause of X(which can only occur if there are latent confounders of Xand some other variable or Yand some other variable; (3) an edge Xo->Y   only if Yis not a direct or indirect cause of X, but Xmay or may not be an indirect cause of Y; (4) an edge X o–o Y   indicates that Xand Y   are dependent no matter what subset of observed variables is conditioned on, but contains no orientation information (X   may be a direct or indirect cause of Y, and Ymay be an indirect cause of X, or there may be a latent common cause of Xand Y.

# Trying some various ML models

In [3]:
#Synthetic data
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import pydot
from IPython.display import SVG
import scipy.stats as ss
import numpy as np
import matplotlib.pyplot as plt


def gen_data():
    #SIZE = 40000
    SIZE = 4000
    a = np.random.normal(0, 1, SIZE)
    b = np.random.normal(0, 1, SIZE)
    c = np.random.normal(0, 1, SIZE)
    d = np.random.normal(0, 1, SIZE)
    e = np.random.normal(0, 1, SIZE)
    f= a + b + c + d + e + np.random.normal(0, 1, SIZE)
    g = f + np.random.normal(0, 1, SIZE)
    g = np.rint(g)
    return pd.DataFrame({'a' : a,'b' : b, 'c' : c, 'd' : d,'e' : e,'f':f, 'g':g})

def gen_data_binomial():
    SIZE = 10000
    a = np.random.binomial(2, 0.5, size=SIZE)
    b = np.random.binomial(2, 0.5, size=SIZE)
    c = np.random.binomial(2, 0.5, size=SIZE)
    d = np.random.binomial(2, 0.5, size=SIZE)
    e = np.random.binomial(2, 0.5, size=SIZE)
    f= a + b + c + d + e + np.random.binomial(2, 0.5, size=SIZE)
    g = f + np.random.binomial(2, 0.5, size=SIZE)
    return pd.DataFrame({'a' : a,'b' : b, 'c' : c, 'd' : d,'e' : e,'f':f, 'g':g})

def gen_data_discrete():
    SIZE = 40000
    a = discrete_gauss(-3,3, SIZE)
    b = discrete_gauss(-3,3, SIZE)
    c = discrete_gauss(-3,3, SIZE)
    d = discrete_gauss(-3,3, SIZE)
    e = discrete_gauss(-3,3, SIZE)
    f= a + b + c + d + e + np.random.binomial(2, 0.5, size=SIZE)
    g = f + discrete_gauss(-3,3, SIZE)
    return pd.DataFrame({'a' : a,'b' : b, 'c' : c, 'd' : d,'e' : e,'f':f, 'g':g})


def discrete_gauss(low, high, samples, std = 20):
    x = np.arange(low, high)
    xU, xL = x + 0.5, x - 0.5 
    prob = ss.norm.cdf(xU, scale = std) - ss.norm.cdf(xL, scale = std)
    prob = prob / prob.sum() #normalize the probabilities so their sum is 1
    nums = np.random.choice(x, size = samples, p = prob)
    return nums



def bar_plot(x_ax, val1, val1std, val2, val2std):
    fig = plt.figure()
    ax = fig.add_subplot(111)

    ## the data
    N = len(x_ax)

    ## necessary variables
    ind = np.arange(N)                # the x locations for the groups
    width = 0.35                      # the width of the bars
    fig.set_size_inches(18.5, 10.5)
    ## the bars
    rects1 = ax.bar(ind, val1, width,
                    color='gray',
                    yerr=val1std,
                    error_kw=dict(elinewidth=2,ecolor='blue'))

    rects2 = ax.bar(ind+width, val2, width,
                        color='blue',
                        #yerr=val2std,
                        error_kw=dict(elinewidth=2,ecolor='gray'))

    # axes and labels
    ax.set_xlim(-width,len(ind)+width)
    #ax.set_ylim(0,45)
    ax.set_ylabel('Percentage')
    ax.set_title('')
    plt.xticks(ind + width / 2, x_ax, rotation=75, size = 14)
    ## add a legend
    ax.legend( (rects1[0], rects2[0]), ('Accuracy', '% Violations') )
    fig.savefig("violations.pdf", bbox_inches='tight')
    plt.show()


In [4]:
# generate our data
df = gen_data()

In [5]:
import configparser
import random
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, average_precision_score
from keras.models import load_model
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, Callback
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import load_model, Model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, \
                        Input, Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.utils import to_categorical
from collections import Counter
import keras.optimizers
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
from keras.utils import plot_model
import glob, os
import tensorflow as tf
import pandas as pd
from random import shuffle

# select your GPU Here
os.environ["CUDA_VISIBLE_DEVICES"]="0" #Comment this line out if you want all GPUS (2 hehe)

# python full-display web browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


def get_model(dense, dropouts, inputs):
    # dense is an ordered list of the number of dense neurons like [1024, 2048, 1024]
    # dropouts is an ordered list of the dropout masks like [0.2, 0.3, 0.4]
    inputs = keras.Input(shape = (inputs,))

    x = keras.layers.Dense(dense[0], activation = 'relu')(inputs)
    x = keras.layers.Dropout(dropouts[0])(x, training=True)
    for den, drop in zip(dense[1:], dropouts[1:]):
        x = keras.layers.Dense(den, activation = 'relu')(x)
        x = keras.layers.Dropout(drop)(x, training=True)
    outputs = keras.layers.Dense(1, activation = 'linear')(x)
    model = keras.Model(inputs, outputs)
    return model


/home/tkyono/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from sklearn.metrics import roc_auc_score, average_precision_score, mean_squared_error, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import pandas as pd
from pycausal import search as s









def gen_data(mean = 0, var = 1, SIZE = 5000):
    a = np.random.gumbel(mean, var, SIZE)
    b = np.random.gumbel(mean, var, SIZE)
    c = np.random.gumbel(mean, var, SIZE)
    d = np.random.gumbel(mean, var, SIZE)
    e = np.random.gumbel(mean, var, SIZE)
    f= a + b + c + d + e + np.random.gumbel(mean, var, SIZE)
    g = f + np.random.gumbel(mean,var, SIZE)
    g = np.rint(g)
    return pd.DataFrame({'a' : a,'b' : b, 'c' : c, 'd' : d,'e' : e,'f':f, 'g':g})

def gen_data(mean = 0, var = 1, SIZE = 5000):
    a = np.random.gumbel(mean, var, SIZE)
    b = np.random.gumbel(mean, var, SIZE)
    c = np.random.gumbel(mean, var, SIZE)
    d = np.random.gumbel(mean, var, SIZE)

    f= a + b + c + d + np.random.gumbel(mean, var, SIZE)
    g = f + np.random.gumbel(mean,var, SIZE)
    
    
    g = np.rint(g)
    e = g + np.random.gumbel(mean,var,SIZE)
    
    return pd.DataFrame({'a' : a,'b' : b, 'c' : c, 'd' : d,'e' : e,'f':f, 'g':g})

def gen_data(mean = 0, var = 1, SIZE = 40000):
    f = np.random.normal(mean, var, SIZE)
    a = f + np.random.normal(mean, var, SIZE)
    b = f + np.random.normal(mean, var, SIZE)
    c = f + np.random.normal(mean, var, SIZE)
    d = f + np.random.normal(mean, var, SIZE)
    e = f + np.random.normal(mean, var, SIZE)
    g = a + b + c + d + e + np.random.normal(mean, var, SIZE)
    g = np.rint(g)
    return pd.DataFrame({'a' : a,'b' : b, 'c' : c, 'd' : d,'e' : e,'f':f, 'g':g})

def gen_data(mean = 0, var = 1, SIZE = 10000):
    a = np.random.normal(mean, var, SIZE)
    b = np.random.normal(mean, var, SIZE)
    c = np.random.normal(mean, var, SIZE)
    d = np.random.normal(mean, var, SIZE)
    e = np.random.normal(mean, var, SIZE)
    f= a + b + c + d + e + np.random.normal(mean, var, SIZE)
    g = f + np.random.normal(mean,var, SIZE)
    g = np.rint(g)
    return pd.DataFrame({'a' : a,'b' : b, 'c' : c, 'd' : d,'e' : e,'f':f, 'g':g})



def get_CG(df, tetrad):
    tetrad.run(algoId = 'gfci', dfs = df, testId = 'sem-bic', scoreId = 'sem-bic', dataType = 'continuous',
           structurePrior = 1.0, samplePrior = 1.0, maxDegree = -1, maxPathLength = -1, 
           completeRuleSetUsed = False, faithfulnessAssumed = True, verbose = True)
    #tetrad.run(algoId = 'gfci', dfs = df, testId = 'bdeu', scoreId = 'bdeu', dataType = 'discrete',
    #       structurePrior = 1.0, samplePrior = 1.0, maxDegree = 3, faithfulnessAssumed = True, verbose = True)


    return tetrad.getTetradGraph()

def get_MB(graph, var, pc):
    parents = set()
    for i in pc.extractTetradGraphEdges(graph):
        if i[-1] == var and i[3:5] == '->':
            parents.add(i[0])
        if i[0] == var and i[3:5] == '->':
            parents.add(i[-1])
    return parents

from pycausal.pycausal import pycausal as pc
from collections import defaultdict
pc = pc()
pc.start_vm(java_max_heap_size = '5000M')
tetrad = s.tetradrunner()

df = gen_data()

means = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
variances = [1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]
def run_models(models, model_names, x, y, num_folds=20):
    violations = np.zeros(len(models))
    violation_mean = np.zeros((len(models), num_folds))
    mean = np.zeros((len(models), num_folds))
    mean2 = np.zeros((len(models), num_folds)) # used to store our secondary metric
    
    skf = StratifiedKFold(n_splits=num_folds)
    fold = 0
    
    metrics_dicts = []
    causal_dicts = []
    for m in models:
        metrics_dicts.append(defaultdict(list))
        causal_dicts.append(defaultdict(list))
    
    for train_index, test_index in skf.split(x, y):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        for idx, mod in enumerate(models):
            keras.backend.clear_session() 
            model = get_model(mod, [0.2] * len(models), 6)
            callbacks = [ModelCheckpoint('temp', verbose=1, monitor='val_loss',save_best_only=True), 
                         EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=1, verbose=1, mode='auto')
                ]
            model.compile(optimizer = optimizers.SGD(lr = 0.001, momentum = 0.9, ), loss='mean_squared_error', metrics = ['mse'])
            model.fit(x_train, y_train, epochs = 20, validation_data = (x_test, y_test), callbacks = callbacks, batch_size = 32)
            model = load_model('temp')
            original_df = pd.DataFrame(x_test, columns = ['a', 'b', 'c', 'd', 'e', 'f'])

            original_targets = pd.DataFrame(y_test, columns = ['g'])
            original_df = original_df.join(original_targets)
            test_df = pd.DataFrame(x_test, columns = ['a', 'b', 'c', 'd', 'e', 'f'])
            test_targets = pd.DataFrame(model.predict(x_test), columns = ['g'])
            test_df = test_df.join(test_targets)
            mean[idx][fold] = mean_squared_error(y_test, model.predict(x_test))
            
            setA = get_MB(get_CG(original_df, tetrad), 'g', pc)
            setB = get_MB(get_CG(test_df, tetrad), 'g', pc)
            
            if setA != {'f'}:
                print("Error in SETA markov blanket")
                setA = {'f'}
            
            violation_mean[idx][fold] = len(setA.difference(setB)) + len(setB.difference(setA))
            if setA != setB:
                print("Violation:", setA , setB)
                violations[idx] += 1
            
            
            # ok at this point we need to check the model on various variances and means
            for m in means:
                for v in variances:
                    perturbed_df = gen_data(mean =m, var = v, SIZE = 2000)
                    y_test2 = perturbed_df['g']
                    x_test2 = perturbed_df[['a', 'b', 'c', 'd', 'e', 'f']]
                    y_pred2 = model.predict(x_test2)
                    #metrics_dicts[idx][str(m) + '_' + str(v)].append(accuracy_score(y_test2, y_pred2))
                    metrics_dicts[idx][str(m) + '_' + str(v)].append(mean_squared_error(y_test2, y_pred2))
                    
                    test_df2 = pd.DataFrame(x_test2, columns = ['a', 'b', 'c', 'd', 'e', 'f'])
                    test_targets2 = pd.DataFrame(model.predict(x_test2), columns = ['g'])
                    test_df2 = test_df2.join(test_targets2)
                    setC = get_MB(get_CG(test_df2, tetrad), 'g', pc)
                    
                    #if model_names[idx] == 'GaussianNB()':
                    #    print(m,v, setC)
                    if setA != setC:
                        causal_dicts[idx][str(m) + '_' + str(v)].append(len(setA.difference(setC)) + len(setC.difference(setA)))
                        causal_dicts[idx][str(m) + '_' + str(v)].append(1)
                    else:
                        causal_dicts[idx][str(m) + '_' + str(v)].append(0)
                    
        fold += 1
    
    print("Violations = ", violations)
    
    metric = []
    metric_err = []
    viol = []
    viol_err = []
    
    #normalize the violations for prettier graphing.
    #also violations are always positive, so just divide by max.
    violation_mean = violation_mean / np.max(violation_mean)
    
    for i in range(len(violations)):
        print("Model_name = ", model_names[i], "Violations = ", violations[i])
        print("Average_violations = ", np.mean(violation_mean[i]), np.std(violation_mean[i]))
        print("Accuracy = ", np.mean(mean[i]), np.std(mean[i]), "MSE = ", np.mean(mean2[i]), np.std(mean2[i]),)
        metric.append(np.mean(mean[i]))
        metric_err.append(np.std(mean[i]))
        viol.append(np.mean(violation_mean[i]))
        
        viol_err.append(np.std(violation_mean[i]))
    print(np.array(metric), 
             np.array(metric_err), 
             np.array(viol), 
             np.array(viol_err))    
    
    bar_plot(model_names, 
             np.array(metric), 
             np.array(metric_err), 
             np.array(viol), 
             np.array(viol_err))
    return metrics_dicts, causal_dicts

X = df[['a', 'b', 'c', 'd', 'e', 'f']].values
y = df['g'].values

'''
models = [LogisticRegression(), 
          Perceptron(),  
          DecisionTreeClassifier(),
          LinearSVC(),
          GaussianNB(),
          LinearDiscriminantAnalysis(),
          RandomForestClassifier(),
          ExtraTreesClassifier(),
          AdaBoostClassifier(),
          BaggingClassifier(),
          GradientBoostingClassifier(),
          MLPClassifier()
         ]
model_names = ['LogisticRegression()', 
          'Perceptron()',  
          'DecisionTreeClassifier()',
          'LinearSVC()',
          'GaussianNB()',
          'LinearDiscriminantAnalysis()',
          'RandomForestClassifier()',
          'ExtraTreesClassifier()', 
          'AdaBoostClassifier()',
          'BaggingClassifier()',
          'GradientBoostingClassifier()',
          'MLPClassifier()'
         ]

'''

models = [#[8,8,4],
          #[16,16,8],
          #[32,32,16],
          #[64,64,32],
          #[128, 128, 64],
          [256, 256, 128],
          [512, 512, 256],
          [1024, 1024, 512],
          [2048, 2048, 1024],

         ]
model_names = [#'m1', 
               #'m2',
               #'m3',
               #'m4', 
               'm5',
               'm6',
               'm7', 
               'm8',
               'm9'
              ]

metrics, violations = run_models(models,model_names, X,y)

/home/tkyono/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/home/tkyono/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=20.
  % (min_groups, self.n_splits)), Warning)


Train on 9488 samples, validate on 512 samples
Epoch 1/20
9488/9488 [==============================] - 3s 267us/step - loss: 1.5325 - mean_squared_error: 1.5325 - val_loss: 1.3198 - val_mean_squared_error: 1.3198

Epoch 00001: val_loss improved from inf to 1.31980, saving model to temp
Epoch 2/20
9488/9488 [==============================] - 1s 103us/step - loss: 1.2399 - mean_squared_error: 1.2399 - val_loss: 1.3773 - val_mean_squared_error: 1.3773

Epoch 00002: val_loss did not improve from 1.31980
Epoch 00002: early stopping
Error in SETA markov blanket
Violation: {'f'} set()


In [ ]:
def heat_plot(x,y,z, clim_low = 0, clim_high = 1):
    fig, ax = plt.subplots()

    cax = ax.scatter(x, y, c=z, s=450, edgecolor='')
    cax.set_clim(clim_low, clim_high)
    ax.set_xlabel('Mean')
    ax.set_ylabel('Variance')
    plt.colorbar(cax)
    plt.show()

for idx, m in enumerate(models):
    print(model_names[idx])
    x = []
    y = []
    z = []
    
    
    
    for k, v in metrics[idx].items():
        x.append(k.split('_')[0])
        y.append(k.split('_')[-1])
        z.append(np.mean(v))
        
    heat_plot(x,y,z, clim_low = 0, clim_high = 10)
    
for idx, m in enumerate(models):
    print(model_names[idx])      
    x = []
    y = []
    z = []
    for k, v in violations[idx].items():
        x.append(k.split('_')[0])
        y.append(k.split('_')[-1])
        z.append(np.sum(v) / len(v))
    print("Causal assurance")
    heat_plot(x,y,z, clim_low = 0, clim_high = 1) 